<a target="_blank" href="https://colab.research.google.com/github/Prindle19/sdsc24/blob/main/01%20-%20EE%20Python%20Intro.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

# Introduction

This rather advanced notebook walks through the process of introducing Google Earth Engine in Python, including computing large raster data and visuailizing it with the geemap library. Additionally, it will demonstrate creating a PDF-format report for a given land plot. It shows the location of the plot and various spectral history figures that illustrate what is in the plot and how the plot changes seasonally and over 40 years. It uses Google Earth Engine along with many common 3rd party Python libraries.

If you need to get set up to use Earth Engine or are unfamiliar with running Earth Engine in a Colab notebook, use the [**Earth Engine Python Quickstart**](https://developers.google.com/earth-engine/guides/quickstart_python) to get going.

_Modified from an example by [Justin Braaten](https://github.com/jdbcode) Originally Presented at the 2024 [Google Geo for Good Sao Paulo](https://earthoutreachonair.withgoogle.com/events/geoforgood24-saopaulo) and [Dublin](https://earthoutreachonair.withgoogle.com/events/geoforgood24-dublin) meetings._

# Setup

Intall packages we need that Colab doesn't have installed by defualt.

In [ ]:
!pip install -q reportlab
!pip install -q -U pillow
!pip install -q cartopy
!pip install -q PyPDF2

Import the libraries we'll need.

In [ ]:
import os
import io
import ee
import geemap
import glob
import requests
import json
import geopandas as gpd
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import datetime
import numpy as np
import matplotlib
import PyPDF2

from PIL import Image, ImageDraw, ImageFont
from matplotlib import pyplot as plt
from matplotlib import cm
from reportlab.pdfgen import canvas
from reportlab.lib.pagesizes import letter
from reportlab.lib.units import inch
from reportlab.lib.utils import ImageReader

Authenticate and initialize to the Earth Engine service.

* Requires a Google Cloud Project registered to use Earth Engine

In [ ]:
ee.Authenticate()
ee.Initialize(project='sdsc24-nyc') # Change to a project where you have EEE access.

# Create a sophisticated cloud-free Sentinel 2 Mosaic

This will *dynamically* create a cloud-free mosaic for an entire year's worth of 10m Sentinel 2 Imagery and display it on a geemap.

1 year of Sentinel 2 Imagery is ~3.5 petabytes

In [ ]:
# Harmonized Sentinel-2 Level 2A collection.
# https://developers.google.com/earth-engine/datasets/catalog/COPERNICUS_S2_SR_HARMONIZED

s2 = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED');

# Cloud Score+ image collection.
# https://developers.google.com/earth-engine/datasets/catalog/GOOGLE_CLOUD_SCORE_PLUS_V1_S2_HARMONIZED
# Note Cloud Score+ is produced from Sentinel-2 Level 1C data
# and can be applied to either L1C or L2A collections.

csPlus = ee.ImageCollection('GOOGLE/CLOUD_SCORE_PLUS/V1/S2_HARMONIZED');

# This function adds a band representing the image timestamp.
def maskThreshold(img):
  return img.updateMask(img.select(QA_BAND).gte(CLEAR_THRESHOLD));

# Link both collections for all images with > 60% CS score (60% of what you'd expect to see on a clear day)
# Reduce the linked collection by the median of all values at each pixel across each band
QA_BAND = 'cs'
CLEAR_THRESHOLD = 0.60
composite = s2.filterDate('2023-01-01', '2023-12-31').linkCollection(csPlus, [QA_BAND]).map(maskThreshold).median();

# Select the R,G, and B bands for visualization and generate an X/Y/Z URL file pattern of the resulting image.
composite = composite.select(['B4', 'B3', 'B2'])
s2Viz = {'min': 0, 'max': 2500};
mapID = composite.getMapId(s2Viz)
xyzURL = mapID['tile_fetcher'].url_format

In [ ]:
# Now display the cloud-free mosaic on a geemap

from ipyleaflet import Map, basemaps, TileLayer, LayersControl
import pandas as pd
import geopandas as gpd

center = (40.791530811684396, -73.95543742118639)
s2Layer = TileLayer(url=xyzURL, opacity=1, name="S2 CS+ 2022")

m = Map(center=center, zoom=11)
m.add(s2Layer)

# Find a point of interest

Create a `geemap.Map` object and display it.
Use the Point marker on the draw control to identify a point to create a report.

In [ ]:
m = geemap.Map()
m.add_basemap('SATELLITE')
m

In [ ]:
# Inspect the point as a client-side object

geometry = m._draw_control.last_geometry
geometry

Create a region of interest. Define the longitude and latitude of the selected location and define a buffer to add to the point and add it to the map.

In [ ]:
# Now create a server-side representation of the point
lon = geometry['coordinates'][0]
lat = geometry['coordinates'][1]
buffer = 250 # meters
ee_region = ee.Geometry.Point(lon, lat).buffer(buffer)
m.add_layer(ee_region, {'color': 'yellow'}, 'ROI')

# Set global and final output variables

Create some folders on the Colab VM and set default file names.

In [ ]:
folder_figures = '/content/figures'
folder_monthly_images = '/content/monthly_images'
folder_annual_images = '/content/annual_images'
folder_pdfs = '/content/pdfs'
folders = [folder_figures, folder_monthly_images, folder_annual_images, folder_pdfs]

small_scale_map = f'{folder_figures}/small_scale_map.png'
large_scale_map = f'{folder_figures}/large_scale_map.png'
single_year = f'{folder_figures}/landsat_large.png'

monthly_nbr = f'{folder_figures}/monthly_nbr.png'
annual_nbr = f'{folder_figures}/annual_nbr.png'

pdf_1 = f'{folder_pdfs}/page_1.pdf'
pdf_2 = f'{folder_pdfs}/page_2.pdf'
pdf_3 = f'{folder_pdfs}/page_3.pdf'
pdf_full = f'{folder_pdfs}/plot_report.pdf'

annual_col = ee.ImageCollection('LANDSAT/COMPOSITES/C02/T1_L2_ANNUAL')
monthly_col = ee.ImageCollection('LANDSAT/COMPOSITES/C02/T1_L2_32DAY')

for folder in folders:
    if not os.path.exists(folder):
        os.makedirs(folder)

# Make location figures

Create a small-scale map of the location.

In [ ]:
plt.figure(figsize=(5, 2.7))
ax = plt.axes(projection=ccrs.PlateCarree())
ax.add_feature(cfeature.LAND, color='lightgrey', zorder=0)
ax.gridlines(draw_labels=True, zorder=1)
ax.scatter(lon, lat, color='blue', zorder=2)
ax.set_global()

plt.savefig(small_scale_map, bbox_inches='tight')
plt.show()

Create a medium-scale map of the location.

In [ ]:
gpd_region = ee.data.computeFeatures({
    'expression': ee.FeatureCollection([ee.Feature(ee_region.bounds())]),
    'fileFormat': 'GEOPANDAS_GEODATAFRAME'
})
buffered_geom = gpd_region.buffer(10)
bounds = buffered_geom.bounds

states_provinces = cfeature.NaturalEarthFeature(
    category='cultural',
    name='admin_1_states_provinces_lines',
    scale='50m',
    facecolor='none')

plt.figure(figsize=(2.7, 2.7))
ax = plt.axes(projection=ccrs.PlateCarree())
ax.add_feature(cfeature.LAND, color='lightgrey', zorder=0)
ax.gridlines(draw_labels=True, zorder=1)
ax.add_feature(cfeature.BORDERS, edgecolor='gray', zorder=2)
ax.add_feature(cfeature.COASTLINE, edgecolor='gray', zorder=3)
ax.add_feature(states_provinces, edgecolor='gray', zorder=4)
ax.scatter(lon, lat, color='blue', zorder=5)
ax.set_extent(
    [bounds['minx'], bounds['maxx'], bounds['miny'], bounds['maxy']],
    crs=ccrs.PlateCarree())
plt.savefig(large_scale_map, bbox_inches='tight')
plt.show()

Define a function to fetch Landsat image chips.

In [ ]:
def prepare_image(image, region_geom, buffer, dim, width, filename):
    '''Makes a visualization image clipped to a region with a plot overlay.'''
    region_geom = ee.Geometry(region_geom)
    region_extract = region_geom.buffer(buffer).bounds()
    region_fc = ee.FeatureCollection(ee.Feature(region_geom))
    region_image = ee.Image().paint(region_fc, width=width).visualize(palette=['FFFFFF'])

    vis = {
        'bands': ['swir1', 'nir', 'green'],
        'min': 0,
        'max': 0.4
    }
    vis_image = (image.visualize(**vis)
        .reproject('EPSG:4326', None, 30)
        .resample('bicubic')
        .blend(region_image)
        .clipToBoundsAndScale(geometry=region_extract, width=dim, height=dim))
    return vis_image

def get_png(image):
    '''Makes a computePixels request from a given image.'''
    return ee.data.computePixels({
        'expression': image,
        'fileFormat': 'PNG'
    })

def save_png(request, filename):
    '''Saves the returned computePixels request to disk.'''
    image = Image.open(io.BytesIO(request))
    image.save(filename)
    return None

def process_image(image, region_geom, buffer, dim, width, filename):
    '''Wraps functions to get an save an image chip for a region.'''
    vis_image = prepare_image(image, region_geom, buffer, dim, width, filename)
    png = get_png(vis_image)
    save_png(png, filename)
    return None

Get an image for the region of interest – last year's annual median.

In [ ]:
current_year = datetime.datetime.now().year

last_year_landsat_mean = (annual_col.filterBounds(ee_region)
    .filterDate(str(current_year - 1), str(current_year))).first()

last_year_landsat_mean

Request the image as a PNG image chip to include in the report.

In [ ]:
process_image(last_year_landsat_mean, ee_region, 5000, 540, 2, single_year)

with Image.open(single_year) as image:
    width, height = image.size
    top = height * 0.15
    bottom = height * 0.85
    cropped_image = image.crop((0, top, width, bottom))
    display(cropped_image)
    cropped_image.save(single_year)

Create a PDF canvas and add text and the location figures to it.

In [ ]:
width, height = letter
page_margin = 0.5 * inch

c = canvas.Canvas(pdf_1, pagesize=letter)

c.setFont('Helvetica-Bold', 16)
c.drawString(page_margin, 740, 'Important Ecological Project')

c.setFont('Helvetica-Bold', 14)
c.drawString(page_margin, 720, 'Region 1')

c.setFont('Helvetica', 12)
c.drawString(page_margin, 690, 'A short description about this marvelous plot.')

c.setFont('Helvetica-Bold', 14)
c.drawString(page_margin, 640, 'Location')

c.drawImage(small_scale_map, page_margin, 450, 320, 173)
c.drawImage(large_scale_map, page_margin + 340, 450, 200, 173)
c.drawImage(single_year, page_margin, page_margin)

c.save()

# Get intra-annual time series

Get a collections of 32-day Landsat composites for the previous year and save image chips for the region of interest.

In [ ]:
#!rm {folder_monthly_images}/*.png

img_col = (monthly_col.filterBounds(ee_region)
    .filterDate(str(current_year-1), str(current_year)))

img_list = img_col.toList(img_col.size())

for i in range(img_col.size().getInfo()):
    img = ee.Image(img_list.get(i))
    date = img.date().format('YYYY-MM-dd').getInfo()
    print(f'Processing: {date}')
    process_image(img, ee_region, 1500, 256, 2, f'{folder_monthly_images}/img_{date}.png')

Display a representative image chip.

In [ ]:
monthly_image_files = sorted(glob.glob(f'{folder_monthly_images}/*.png'))

with Image.open(monthly_image_files[6]) as image:
    display(image)

Start the second page of the report. It starts with the 32-day image chips.

In [ ]:
width, height = letter
page_margin = 0.5 * inch

image_margin_x = 0.11 * inch
image_margin_y = 0.25 * inch

max_image_width = 1.77 * inch
max_image_height = 1.77 * inch
x_offset = page_margin
y_offset = height - page_margin - 0.75 * inch  # Reduced initial spacing

# Create a new PDF canvas
c = canvas.Canvas(pdf_2, pagesize=letter)

c.setFont('Helvetica-Bold', 14)
c.drawString(page_margin, 740, 'Monthly spectral history for previous year and 10-year comparison')
c.setFont('Helvetica', 10)

# Function to add an image to the canvas and wrap if necessary
def add_image(image_path):
    global x_offset, y_offset

    # Open the image using Pillow
    img = Image.open(image_path)

    # Resize image while preserving aspect ratio
    img.thumbnail((max_image_width, max_image_height))
    image_width, image_height = img.size

    # Check if the image and label fit on the current page
    if x_offset + image_width + image_margin_x + page_margin > width:
        x_offset = page_margin
        y_offset -= max_image_height + image_margin_y + 0.15 * inch  # Reduced spacing

    if y_offset - image_height - 0.15 * inch - page_margin < 0:
        c.showPage()
        x_offset = page_margin
        y_offset = height - page_margin - 0.15 * inch

    # Add label above the image
    c.drawString(
        x_offset, y_offset, os.path.basename(image_path).replace('img_', '').replace('.png', ''))

    # Draw the image on the canvas
    c.drawImage(image_path, x_offset, y_offset - image_height - 0.15*inch, image_width, image_height)

    # Update x_offset for the next image
    x_offset += image_width + image_margin_x


for image_path in monthly_image_files:
  add_image(image_path)


Get a DataFrame that includes information to generate an inter-annual time series chart for the previous 11 years.

In [ ]:
def get_region_mean(region):
    '''Returns a function to compute the mean of a region.'''
    def region_mean(img):
        '''Computes the mean of a region.'''
        region_stats = img.reduceRegion(
            geometry=region, reducer=ee.Reducer.mean(), scale=30)
        return ee.Feature(None, region_stats).set(
            {'month': img.date().get('month'), 'year': img.date().get('year')})
    return region_mean

img_col = (monthly_col.filterBounds(ee_region)
    .filterDate(str(current_year-11), str(current_year)))

monthly_images = img_col.map(get_region_mean(ee_region))

df = ee.data.computeFeatures(
    {'expression': monthly_images, 'fileFormat': 'PANDAS_DATAFRAME'})

df['nbr'] = (df['nir'] - df['swir1']) / (df['nir'] + df['swir1'])
df

Make a chart from the 11-year inter-annual time series DataFrame.

In [ ]:
plot_width = width - 2 * page_margin
plot_height = height - 2 * page_margin

plot_height = 2.8
fig, ax = plt.subplots(figsize=(plot_width / inch, plot_height))

viridis = cm.get_cmap('viridis', 10)
colors = viridis(np.linspace(0, 1, 10))
hex_colors = [matplotlib.colors.rgb2hex(color) for color in colors]

for i, year in enumerate(df['year'].unique()):
    df_subset = df[df['year'] == year]
    if i == len(df['year'].unique()) -1:
        ax.plot(df_subset['month'], df_subset['nbr'], label=year, color='orange', linewidth=3)
    else:
        ax.plot(df_subset['month'], df_subset['nbr'], label=year, color=hex_colors[i])

ax.set_ylim(-0.3, 0.8)
ax.set_xlabel('Month')
ax.set_ylabel('NBR')
ax.set_title('')
ax.set_xticks(df['month'].unique())
ax.legend()
plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))

plt.savefig(monthly_nbr, bbox_inches='tight')
plt.show()

Add the chart to the 2nd page of the PDF report.

In [ ]:
c.drawImage(monthly_nbr, page_margin, page_margin, plot_width, 200)
c.save()

# Get annual time series

Get a collection of annual Landsat composites for the previous year and get the DataFrame of the regional mean for each image.

In [ ]:
img_col = (annual_col.filterBounds(ee_region)
    .filterDate('1984-01-01', f'{str(current_year)}-01-01'))

annual_images = img_col.map(get_region_mean(ee_region))

df = ee.data.computeFeatures(
    {'expression': annual_images, 'fileFormat': 'PANDAS_DATAFRAME'})

df['nbr'] = (df['nir'] - df['swir1']) / (df['nir'] + df['swir1'])
df

Create a time series chart from the annual time series DataFrame.

In [ ]:
df['r'] = df['swir1'] * (255 / 0.4)
df['g'] = df['nir'] * (255 / 0.4)
df['b'] = df['green'] * (255 / 0.4)
rgb_values = df[['r', 'g', 'b']].values.astype(int) / 255

plot_height = 3
fig, ax = plt.subplots(figsize=(plot_width / inch, plot_height))

ax.plot(df['year'], df['nbr'], color='black', zorder=1)
ax.scatter(df['year'], df['nbr'], c=rgb_values, s=100, zorder=2)
ax.set_ylim(-0.3, 0.8)
ax.set_xticks(df['year'])
ax.set_xticklabels(df['year'], rotation=90)

ax.set_xlabel('Year')
ax.set_ylabel('NBR')

plt.savefig(annual_nbr, bbox_inches='tight')
plt.show()

Get a time series of image chips from the annual composite collection.

In [ ]:
img_list = img_col.toList(img_col.size())

for i in range(img_col.size().getInfo()):
    img = ee.Image(img_list.get(i))
    date = img.date().format('YYYY').getInfo()
    print(f'Processing: {date}')
    process_image(img, ee_region, 1500, 256, 2, f'{folder_annual_images}/img_{date}.png')

Add the annual time series chart and the annual image chips to the 3rd PDF file.

In [ ]:
c = canvas.Canvas(pdf_3, pagesize=letter)

c.setFont('Helvetica-Bold', 14)
c.drawString(page_margin, 740, 'Annual spectral history')
c.setFont('Helvetica', 10)

c.drawImage(annual_nbr, page_margin, 475, plot_width, 245)

y_offset = height - page_margin - 2.5 * inch

annual_image_files = sorted(glob.glob(f'{folder_annual_images}/*.png'))

for image_path in annual_image_files:
  add_image(image_path)

c.save()

# Put it all together

Combine the 3 individual PDF files into a single-file report.

In [ ]:
def merge_pdfs(paths, output):
  merger = PyPDF2.PdfMerger()
  for path in paths:
    with open(path, 'rb') as f:
      merger.append(f)
  with open(output, 'wb') as f:
    merger.write(f)

pdfs = sorted(glob.glob(f'{folder_pdfs}/*.pdf'))
merge_pdfs(pdfs, pdf_full)